In [ ]:
import sys
sys.path.append('../')
sys.path.append('../tests/python-support/')

%matplotlib notebook

In [ ]:
import matplotlib
import experiments
import itertools
import util
import numpy as np
import json
from collections import defaultdict

from pprint import pprint

# Configuration

In [ ]:
# Whether to only run the offline part of the experiment,
# or both offline and online
offline_only = False

# Whether to import/export homogenized basis
import_basis = True
export_basis = False

# Names of experiments to run
experiment_names = [ "homogenized_l_shaped" ]

# Offline parameters. 
# Use integer exponents so that we can have reliable filenames
h_base = 0.5
h_exponent = [2, 3, 4]
oversampling = [ 2 ]

# Online parameters
end_time = [0.5]
sample_count = [ 401 ]
integrator = [ "lumped_leapfrog", "iterative_leapfrog" ]

In [ ]:
def make_offline_param(h_exponent, oversampling):
    p = {
        'mesh_resolution': h_base ** h_exponent,
        'oversampling': oversampling
    }
    basis_file = 'basis_{}_{}.h5'.format(h_exponent, oversampling)
    if import_basis:
        p['basis_import_file'] = basis_file
    if export_basis:
        p['basis_export_file'] = basis_file
    return p

def make_online_param(end_time, sample_count, integrator):
    return {
        'end_time': end_time,
        'sample_count': sample_count,
        'integrator': integrator,
        'load_quadrature_strength': 2
    }

def aggregate_offline_params():
    offline_param_product = itertools.product(h_exponent, oversampling)
    return [ make_offline_param(h_exp, ovs) for (h_exp, ovs) in offline_param_product]

def aggregate_online_params():
    online_param_product = itertools.product(end_time, sample_count, integrator)
    return [ make_online_param(e, s, i) for (e, s, i) in online_param_product ]

def aggregate_input():
    offline_params = aggregate_offline_params()
    online_params = aggregate_online_params()
    if offline_only:
        return [{ 
                    'experiment': experiment,
                    'offline': offline
                }
                for (experiment, offline) 
                in itertools.product(experiment_names, offline_params)
               ]
    else:
        return [{ 
                    'experiment': experiment,
                    'offline': offline,
                    'online': online 
                }
                for (experiment, offline, online) 
                in itertools.product(experiment_names, offline_params, online_params)
               ]
    

experiment_input = aggregate_input()

In [ ]:
# Run experiments and collect results
results = experiments.run_experiments(experiment_input)

In [ ]:
# Uncomment to inspect raw results
pprint(results)

# Analysis of results

In [ ]:
def is_successful(result):
    return 'error' not in result

def key(experiment):
    name = experiment['experiment']
    offline_params = experiment['offline']['parameters']
    oversampling = offline_params['oversampling']
    if offline_only:
        return (name, "no integration", oversampling)
    else:
        online_params = experiment['online']['parameters']
        integrator = online_params['integrator']
        return (name, integrator, oversampling)

success_results = [ result for result in results if is_successful(result) ]
failure_results = [ result for result in results if not is_successful(result) ]

# TODO: Distinguish failures

def flatten_timing(timing):
    flattened_timing = defaultdict(list)
    for experiment_timing in timing:
        for k, v in experiment_timing.items():
            flattened_timing[k].append(v)
    return flattened_timing

def print_timing(flattened_timing):
    timing_pad = len(max(flattened_timing.keys(), key = lambda s: len(s)))

    for timing_name, values in sorted(flattened_timing.items(), key=lambda pair: pair[0]):
        padded_name = str("{}:".format(timing_name).ljust(timing_pad + 1))
        formatted_values = ", ".join([ "{:10.4g}".format(val) for val in values])
        print("{} [ {} ]".format(padded_name, formatted_values))

success_results = sorted(success_results, key = key)
grouped_results = itertools.groupby(success_results, key = key)
for (name, integrator, oversampling), result in grouped_results:
    result = list(result)
    offline_results = [ result['offline']['result'] for result in result ]
    num_dof = [ result['mesh_details']['num_vertices'] for result in offline_results ]
    
    offline_timings = [ result['offline']['result']['timing'] for result in result ]
    offline_timings = flatten_timing(offline_timings)
    
    print("=============================================")
    print("Experiment name: {name}".format(name=name))
    print("Integrator:      {}".format(integrator))
    print("Oversampling:    {}".format(oversampling))
    print("Dof:             {}".format(num_dof))
    
    if not offline_only:
        online_results = [ result['online']['result'] for result in result ]
        l2_error = [ result['error_summary']['l2'] for result in online_results ]
        h1_error = [ result['error_summary']['h1'] for result in online_results ]
        h1_semi_error = [ result['error_summary']['h1_semi'] for result in online_results ]
        l2_slope = util.estimate_slope(num_dof, l2_error)
        h1_slope = util.estimate_slope(num_dof, h1_error)
        
        online_timings = [ result['online']['result']['timing'] for result in result ]
        online_timings = flatten_timing(online_timings)
        print("H1 semi:  {}".format(h1_semi_error))
        print("H1:       {}".format(h1_error))
        print("L2:       {}".format(l2_error))
        print("H1 slope: {}".format(h1_slope))
        print("L2 slope: {}".format(l2_slope))
        print("")
        print("Timing (online):")
        print("---------------------------------------------")
        print_timing(online_timings)

    
    print("")
    print("Timing (offline):")
    print("---------------------------------------------")
    print_timing(offline_timings)
    print("=============================================")
    print("")